# Food Delivery Cost and Profitability Analysis

### IMPORTING DATA FROM THE CSV FILE

In [ ]:
# Loading the csv file
import pandas
food_orders = pandas.read_csv("food_orders_new_delhi.csv")
food_orders.head()

In [ ]:
# Showcasing the brief intro of the dataframe
food_orders.info()   # type: ignore

### DATA PREPARATION

In [ ]:
from datetime import datetime

In [ ]:
# Convert data and time columns to datetime
food_orders['Order Date and Time'] = pd.to_datetime(food_orders['Order Date and Time'], format='%d-%m-%Y %H:%M') 
food_orders['Delivery Date and Time'] = pd.to_datetime(food_orders['Delivery Date and Time'], format='%d-%m-%Y %H:%M')

In [ ]:
# Filling the null values in the 'Discounts and Offers' with '0'
food_orders['Discounts and Offers'] = food_orders['Discounts and Offers'].fillna('0')

In [ ]:
# Extract numeric values from the 'Discounts and Offers' string
def extract_discount(discount_str):
    # Fixed amount off
    if 'off' in discount_str:
        return float(discount_str.split(' ')[0])
    # Percentage off
    elif '%' in discount_str:
        return float(discount_str.split('%')[0])
    # No discount
    else:
        return 0.0

In [ ]:
# Create Discount Percentage column
food_orders['Discount Percentage'] = food_orders['Discounts and Offers'].apply(lambda x: extract_discount(x))

# Calculate Discount Amount
discount_condition = food_orders['Discount Percentage'] > 1
food_orders['Discount Amount'] = food_orders['Order Value'] * food_orders['Discount Percentage'] / 100 * discount_condition

# Handle fixed discounts
food_orders.loc[food_orders['Discount Percentage'] <= 1, 'Discount Amount'] = food_orders['Discount Percentage']

# Print desired columns and data types
food_orders[['Order Value', 'Discounts and Offers', 'Discount Percentage', 'Discount Amount']].head()

In [ ]:
# Checking the data types of all columns
food_orders.dtypes

### PROFITABILITY ANALYSIS

In [ ]:
# calculate total costs and revenue per order
food_orders['Total Costs'] = food_orders['Delivery Fee'] + food_orders['Payment Processing Fee'] + food_orders['Discount Amount']
food_orders['Revenue'] = food_orders['Commission Fee']
food_orders['Profit'] = food_orders['Revenue'] - food_orders['Total Costs']

In [ ]:
# aggregate data to get overall metrics
total_orders = food_orders.shape[0]
total_revenue = food_orders['Revenue'].sum()
total_costs = food_orders['Total Costs'].sum()
total_profit = food_orders['Profit'].sum()

In [ ]:
# Overall Metrics
overall_metrics = {
    "Total Orders": total_orders,
    "Total Revenue": total_revenue,
    "Total Costs": total_costs,
    "Total Profit": total_profit
}

overall_metrics

### DATA VISUALISATION

In [ ]:
# Profit Distribution per order 
import matplotlib
from matplotlib import pyplot as plt 

# histogram of profits per order
plt.figure(figsize=(10, 6))
plt.hist(food_orders['Profit'], bins=50, color='skyblue', edgecolor='black')
plt.title('Profit Distribution per Order in Food Delivery')
plt.xlabel('Profit')
plt.ylabel('Number of Orders')
plt.axvline(food_orders['Profit'].mean(), color='red', linestyle='dashed', linewidth=1)
plt.show()

In [ ]:
# pie chart for the proportion of total costs
costs_breakdown = food_orders[['Delivery Fee', 'Payment Processing Fee', 'Discount Amount']].sum()
plt.figure(figsize=(7, 7))
plt.pie(costs_breakdown, labels=costs_breakdown.index, autopct='%1.1f%%', startangle=140, colors=['tomato', 'gold', 'lightblue'])
plt.title('Proportion of Total Costs in Food Delivery')
plt.show()

In [ ]:
# bar chart for total revenue, costs, and profit
totals = ['Total Revenue', 'Total Costs', 'Total Profit']
values = [total_revenue, total_costs, total_profit]

plt.figure(figsize=(8, 6))
plt.bar(totals, values, color=['gold', 'green', 'red'])
plt.title('Total Revenue, Costs, and Profit')
plt.ylabel('Amount (INR)')
plt.show()

### FINDING NEW AVERAGES OF DISCOUNTS AND CHARGING COMMISSIONS

In [ ]:
# filter the dataset for profitable orders
profitable_orders = food_orders[food_orders['Profit'] > 0]

In [ ]:
# calculate the average commission percentage for profitable orders
profitable_orders['Commission Percentage'] = (profitable_orders['Commission Fee'] / profitable_orders['Order Value']) * 100

In [ ]:
# calculate the average discount percentage for profitable orders
8
profitable_orders['Effective Discount Percentage'] = (profitable_orders['Discount Amount'] / profitable_orders['Order Value']) * 100

In [ ]:
# calculate the new averages
new_avg_commission_percentage = profitable_orders['Commission Percentage'].mean()
new_avg_discount_percentage = profitable_orders['Effective Discount Percentage'].mean()

In [ ]:
print(new_avg_commission_percentage, new_avg_discount_percentage)

### PROFITABILITY ANALYSIS WITH PROFITABLE VALUES 

In [ ]:
# simulate profitability with recommended discounts and commissions
recommended_commission_percentage = 30.0  # 30%
recommended_discount_percentage = 6.0    # 6%

In [ ]:
# calculate the simulated commission fee and discount amount using recommended percentages
food_orders['Simulated Commission Fee'] = food_orders['Order Value'] * (recommended_commission_percentage / 100)
food_orders['Simulated Discount Amount'] = food_orders['Order Value'] * (recommended_discount_percentage / 100)

In [ ]:
# recalculate total costs and profit with simulated values
food_orders['Simulated Total Costs'] = (food_orders['Delivery Fee'] + food_orders['Payment Processing Fee'] + food_orders['Simulated Discount Amount'])

In [ ]:
food_orders['Simulated Profit'] = (food_orders['Simulated Commission Fee'] - food_orders['Simulated Total Costs'])

In [ ]:
print(food_orders)

### PROFITABILITY ANALYSIS

In [ ]:
# Calculate total costs and revenue per order
food_orders['New Total Costs'] = food_orders['Delivery Fee'] + food_orders['Payment Processing Fee'] + food_orders['Simulated Discount Amount']
food_orders['New Revenue'] = food_orders['Simulated Commission Fee']
food_orders['New Profit'] = food_orders['New Revenue'] - food_orders['Simulated Total Costs']

In [ ]:
# Aggregate data to get overall metrics
new_total_revenue = food_orders['New Revenue'].sum()
new_total_costs = food_orders['New Total Costs'].sum()
new_total_profit = food_orders['New Profit'].sum()

In [ ]:
# Overall Metrics
overall_metrics = {
    "Total Orders": total_orders,
    "Total Revenue": new_total_revenue, #type: ignore 
    "Total Costs": new_total_costs,     #type: ignore 
    "Total Profit": new_total_profit    #type: ignore 
}

print(overall_metrics)

### VISUALISATION

In [ ]:
# Histogram of profits per order
plt.figure(figsize=(10, 6))
plt.hist(food_orders['Simulated Profit'], bins=50, color='skyblue', edgecolor='black')
plt.title('Profit Distribution per Order in Food Delivery')
plt.xlabel('Profit')
plt.ylabel('Number of Orders')
plt.axvline(food_orders['Simulated Profit'].mean(), color='red', linestyle='dashed', linewidth=1)
plt.show()

In [ ]:
# Pie chart for the proportion of total costs
costs_breakdown = food_orders[['Delivery Fee', 'Payment Processing Fee', 'Simulated Discount Amount']].sum()
plt.figure(figsize=(7, 7))
plt.pie(costs_breakdown, labels=costs_breakdown.index, autopct='%1.1f%%', startangle=140, colors=['tomato', 'gold', 'lightblue'])
plt.title('Proportion of Total Costs in Food Delivery')
plt.show()

In [ ]:
# Bar chart for total revenue, costs, and profit
totals = ['Simulated Total Revenue', 'Simulated Total Costs', 'Simulated Total Profit']
values = [new_total_revenue, new_total_costs, new_total_profit]  # type: ignore

plt.figure(figsize=(8, 6))
plt.bar(totals, values, color=['gold', 'green', 'red'])
plt.title('Simulated Total Revenue, Simulated Total Costs, and Simulated Total Profit')
plt.ylabel('Amount (INR)')
plt.show()

## Visualizing Comparison

In [ ]:
# Visualizing the comparison
import seaborn as sns

plt.figure(figsize=(14, 7))

# Actual profitability
sns.kdeplot(food_orders['Profit'], label='Actual Profitability', fill=True, alpha=0.5, linewidth=2)

# Simulated profitability
sns.kdeplot(food_orders['Simulated Profit'], label='Estimated Profitability with Recommended Rates', fill=True, alpha=0.5, linewidth=2)

plt.title('Comparison of Profitability in Food Delivery: Actual vs. Recommended Discounts and Commissions')
plt.xlabel('Profit')
plt.ylabel('Density')
plt.legend(loc='upper left')
plt.show()